In [1]:
import pandas as pd
import itertools
import seaborn as sns

In [2]:
import pybedtools as pbt

In [3]:
contacts_in_significant_pairs = pd.read_csv("../../results/csv/WE_positions_std.csv", sep = "\t")
contacts_in_significant_pairs.head()

,chrom,pos1,strand1,h1,pos2,strand2,h2,transh,pos1-1,pos2-1,interactionID,replicate,dist,bin_name1,bin_name2,bin pair type,std1,std2,num_contacts
0,chr2L,10000460,+,DGRP-57,10002717,-,DGRP-57,False,10000459,10002716,93553,Rep1,2257,chr2L:10000000-10002500,chr2L:10002500-10005000,transDOWN,732.88617,639.025214,210
1,chr2L,10000646,DGRP-57,DGRP-57,10003143,DGRP-57,DGRP-57,False,10000645,10003142,158488,Rep1,2497,chr2L:10000000-10002500,chr2L:10002500-10005000,transDOWN,732.88617,639.025214,210
2,chr2L,10001136,DGRP-57,DGRP-57,10004253,DGRP-57,DGRP-57,False,10001135,10004252,186509,Rep1,3117,chr2L:10000000-10002500,chr2L:10002500-10005000,transDOWN,732.88617,639.025214,210
3,chr2L,10000483,-,DGRP-57,10003916,+,DGRP-57,False,10000482,10003915,448998,Rep1,3433,chr2L:10000000-10002500,chr2L:10002500-10005000,transDOWN,732.88617,639.025214,210
4,chr2L,10002350,DGRP-57,DGRP-57,10004194,DGRP-57,DGRP-57,False,10002349,10004193,901009,Rep1,1844,chr2L:10000000-10002500,chr2L:10002500-10005000,transDOWN,732.88617,639.025214,210


In [4]:
significant = pd.read_csv("../../results/csv/WE_diff_bin_pairs.csv", sep = "\t", index_col = 0)
significant["comparison"] = significant.index.str.split("_chr").str[0]
significant.head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,comparison
DGRP-57_locus1_chr2L:10000000-10002500_chr2L:10002500-10005000,11.328060,-3.026789,1.058510,-2.859482,0.004243,0.045783,DGRP-57_locus1
DGRP-57_locus1_chr2L:10025000-10027500_chr2L:10027500-10030000,4.884034,-4.421116,1.483977,-2.979234,0.002890,0.044235,DGRP-57_locus1
DGRP-57_locus1_chr2L:10190000-10192500_chr2L:10197500-10200000,3.908042,-4.106179,1.494520,-2.747490,0.006005,0.046718,DGRP-57_locus1
DGRP-57_locus1_chr2L:10210000-10212500_chr2L:10217500-10220000,3.973403,-4.134231,1.499326,-2.757392,0.005826,0.046499,DGRP-57_locus1
DGRP-57_locus1_chr2L:10212500-10215000_chr2L:10215000-10217500,11.605186,-2.394369,0.878435,-2.725720,0.006416,0.046838,DGRP-57_locus1


# Prepare bedpe files

In [5]:
def select_color(row):
    colors = {"DGRP-57_DGRP-57" : "136,136,136",
              "DGRP-439_DGRP-439" : "136,136,136",
              "DGRP-57_DGRP-439" : "24,82,73",
              "DGRP-439_DGRP-57" : "24,82,73"}
    
    return colors[row["h1"] + "_" + row["h2"]]

def prepare_bedpe_files(contacts_df, selected_bins):    
    analyzed_haplotype = selected_bins.index.str.split("_").str[0].values[0]
    analyzed_locus = selected_bins.index.str.split("_").str[1].values[0]
    selected_bins["bin_pair"] = selected_bins.index.str.split("_").str[2] + "_" + selected_bins.index.str.split("_").str[3]
    if analyzed_locus == "locus1":
        check_haplotype_column = "h1"
    else:
        check_haplotype_column = "h2"
    
    #select contacts in which the analyzed locus_haplotype participates 
    #and which join selected bin pairs
    diff_contacts = contacts_df[(contacts_df["bin_name1"] + "_" + contacts_df["bin_name2"]).isin(selected_bins["bin_pair"]) &
                                (contacts_df[check_haplotype_column] == analyzed_haplotype)]
    
    
    diff_contacts = diff_contacts.rename(columns = {"chrom":"chr1", "pos1":"x2", "pos2":"end2"})
    diff_contacts["x1"] = diff_contacts["x2"] - 1
    diff_contacts["start2"] = diff_contacts["end2"] - 1
    diff_contacts["chrom2"] = diff_contacts["chr1"]
    diff_contacts["name"] = analyzed_haplotype + "_" + analyzed_locus + "_" + diff_contacts["bin_name1"] + "_" + diff_contacts["bin_name2"]
    diff_contacts["score"] = 1
    diff_contacts["strand1"] = "."
    diff_contacts["strand2"] = "."
    
    diff_contacts["color"] = diff_contacts.apply(select_color, axis=1)
    
    desired_columns = ["chr1", "x1", "x2", "chrom2", "start2", "end2", 
                        "name", "score", "strand1", "strand2", "color"]
    diff_contacts_cis = diff_contacts[diff_contacts["transh"] == False].reindex(desired_columns, axis=1)
    diff_contacts_trans = diff_contacts[diff_contacts["transh"] == True].reindex(desired_columns, axis=1)
    
    return diff_contacts_cis, diff_contacts_trans

In [6]:
contacts_in_significant_pairs.head()

,chrom,pos1,strand1,h1,pos2,strand2,h2,transh,pos1-1,pos2-1,interactionID,replicate,dist,bin_name1,bin_name2,bin pair type,std1,std2,num_contacts
0,chr2L,10000460,+,DGRP-57,10002717,-,DGRP-57,False,10000459,10002716,93553,Rep1,2257,chr2L:10000000-10002500,chr2L:10002500-10005000,transDOWN,732.88617,639.025214,210
1,chr2L,10000646,DGRP-57,DGRP-57,10003143,DGRP-57,DGRP-57,False,10000645,10003142,158488,Rep1,2497,chr2L:10000000-10002500,chr2L:10002500-10005000,transDOWN,732.88617,639.025214,210
2,chr2L,10001136,DGRP-57,DGRP-57,10004253,DGRP-57,DGRP-57,False,10001135,10004252,186509,Rep1,3117,chr2L:10000000-10002500,chr2L:10002500-10005000,transDOWN,732.88617,639.025214,210
3,chr2L,10000483,-,DGRP-57,10003916,+,DGRP-57,False,10000482,10003915,448998,Rep1,3433,chr2L:10000000-10002500,chr2L:10002500-10005000,transDOWN,732.88617,639.025214,210
4,chr2L,10002350,DGRP-57,DGRP-57,10004194,DGRP-57,DGRP-57,False,10002349,10004193,901009,Rep1,1844,chr2L:10000000-10002500,chr2L:10002500-10005000,transDOWN,732.88617,639.025214,210


In [7]:
for name, group in contacts_in_significant_pairs.groupby(["h1", "h2"]):
    print(name[0] + "_" + name[1])
    group_to_save = group.rename(columns = {"chrom":"chr1", "pos1":"x2", "pos2":"end2"})
    group_to_save["x1"] = group_to_save["x2"] - 1
    group_to_save["start2"] = group_to_save["end2"] - 1
    group_to_save["chrom2"] = group_to_save["chr1"]
    group_to_save["name"] = name[0] + "_" + name[1]
    group_to_save["score"] = 1
    group_to_save["strand1"] = "."
    group_to_save["strand2"] = "."
    
    group_to_save["color"] = group_to_save.apply(select_color, axis=1)
    
    desired_columns = ["chr1", "x1", "x2", "chrom2", "start2", "end2", 
                        "name", "score", "strand1", "strand2", "color"]
    group_to_save = group_to_save.reindex(desired_columns, axis=1)
    group_to_save.to_csv("../../results/bed/WE_%s_merged_replicates.bedpe" % (name[0] + "_" + name[1]), index = False, sep = "\t")

DGRP-439_DGRP-439
DGRP-439_DGRP-57
DGRP-57_DGRP-439
DGRP-57_DGRP-57


In [8]:
rep1 = contacts_in_significant_pairs[contacts_in_significant_pairs["replicate"] == "Rep1"]
rep2 = contacts_in_significant_pairs[contacts_in_significant_pairs["replicate"] == "Rep2"]

for num, df in [("1", rep1), ("2", rep2)]:
    for comparison, group in significant.groupby("comparison"):
        cis, trans = prepare_bedpe_files(df, group)
        cis.to_csv("../../results/bed/WE_%s_cis_Rep%s.bedpe" % (comparison, num), index = False, sep = "\t")
        trans.to_csv("../../results/bed/WE_%s_trans_Rep%s.bedpe" % (comparison, num), index = False, sep = "\t")

## merge replicates

In [9]:
for comparison, group in significant.groupby("comparison"):
    cis, trans = prepare_bedpe_files(contacts_in_significant_pairs, group)
    cis.to_csv("../../results/bed/WE_%s_cis_merged_replicates.bedpe" % (comparison), index = False, sep = "\t")
    trans.to_csv("../../results/bed/WE_%s_trans_merged_replicates.bedpe" % (comparison), index = False, sep = "\t")

In [10]:
!head ../../results/bed/WE_DGRP-439_locus1_cis_merged_replicates.bedpe

chr1	x1	x2	chrom2	start2	end2	name	score	strand1	strand2	color
chr2L	10027842	10027843	chr2L	10031736	10031737	DGRP-439_locus1_chr2L:10027500-10030000_chr2L:10030000-10032500	1	.	.	136,136,136
chr2L	10027737	10027738	chr2L	10031595	10031596	DGRP-439_locus1_chr2L:10027500-10030000_chr2L:10030000-10032500	1	.	.	136,136,136
chr2L	10029195	10029196	chr2L	10032375	10032376	DGRP-439_locus1_chr2L:10027500-10030000_chr2L:10030000-10032500	1	.	.	136,136,136
chr2L	10028373	10028374	chr2L	10030667	10030668	DGRP-439_locus1_chr2L:10027500-10030000_chr2L:10030000-10032500	1	.	.	136,136,136
chr2L	10028256	10028257	chr2L	10030221	10030222	DGRP-439_locus1_chr2L:10027500-10030000_chr2L:10030000-10032500	1	.	.	136,136,136
chr2L	10028918	10028919	chr2L	10030089	10030090	DGRP-439_locus1_chr2L:10027500-10030000_chr2L:10030000-10032500	1	.	.	136,136,136
chr2L	10028474	10028475	chr2L	10030062	10030063	DGRP-439_locus1_chr2L:10027500-10030000_chr2L:10030000-10032500	1	.	.	136,136,136
chr2L	10028256	10028257	chr

In [11]:
!head ../../results/bed/WE_DGRP-439_locus2_cis_merged_replicates.bedpe

chr1	x1	x2	chrom2	start2	end2	name	score	strand1	strand2	color
chr2L	10026712	10026713	chr2L	10029943	10029944	DGRP-439_locus2_chr2L:10025000-10027500_chr2L:10027500-10030000	1	.	.	136,136,136
chr2L	10027032	10027033	chr2L	10029473	10029474	DGRP-439_locus2_chr2L:10025000-10027500_chr2L:10027500-10030000	1	.	.	136,136,136
chr2L	10027155	10027156	chr2L	10028703	10028704	DGRP-439_locus2_chr2L:10025000-10027500_chr2L:10027500-10030000	1	.	.	136,136,136
chr2L	10026983	10026984	chr2L	10028964	10028965	DGRP-439_locus2_chr2L:10025000-10027500_chr2L:10027500-10030000	1	.	.	136,136,136
chr2L	10026918	10026919	chr2L	10027922	10027923	DGRP-439_locus2_chr2L:10025000-10027500_chr2L:10027500-10030000	1	.	.	136,136,136
chr2L	10027044	10027045	chr2L	10028401	10028402	DGRP-439_locus2_chr2L:10025000-10027500_chr2L:10027500-10030000	1	.	.	136,136,136
chr2L	10027272	10027273	chr2L	10028938	10028939	DGRP-439_locus2_chr2L:10025000-10027500_chr2L:10027500-10030000	1	.	.	136,136,136
chr2L	10026984	10026985	chr

In [12]:
def select_colors(logFC):
    if logFC > 0:
        return "202,0,32"
    else:
        return "5,113,176"

def prepare_bedpe_files_foldchange(selected):
    selected_bins = selected.copy()
    selected_bins["bin1"] = selected_bins.index.str.split('_').str[2]
    selected_bins["bin2"] = selected_bins.index.str.split('_').str[3]
    selected_bins["chr1"] = selected_bins["bin1"].str.split(':').str[0]
    selected_bins["x1"] = selected_bins["bin1"].str.split(':').str[1].str.split("-").str[0]
    selected_bins["x2"] = selected_bins["bin1"].str.split(':').str[1].str.split("-").str[1]
    selected_bins["chrom2"] = selected_bins["bin2"].str.split(':').str[0]
    selected_bins["start2"] = selected_bins["bin2"].str.split(':').str[1].str.split("-").str[0]
    selected_bins["end2"] = selected_bins["bin2"].str.split(':').str[1].str.split("-").str[1]
    
    selected_bins["name"] = selected_bins.index
    selected_bins["score"] = selected_bins["padj"]
    selected_bins["strand1"] = "."
    selected_bins["strand2"] = "."
    selected_bins["color"] = selected_bins.log2FoldChange.apply(lambda x: select_colors(x))
    selected_bins = selected_bins.reindex(["chr1", "x1", "x2", "chrom2", "start2", "end2", 
                                           "name", "score", "strand1", "strand2", "color"], axis=1)
    
    return selected_bins

In [13]:
for comparison, group in significant.groupby("comparison"):
    fc_colored = prepare_bedpe_files_foldchange(group)
    fc_colored.to_csv("../../results/bed/WE_%s_log2FC_colored.bedpe" % comparison, index = False, sep = "\t")

# Tracks for PnM

In [14]:
contacts_in_significant_pairsPnM = pd.read_csv("../../results/csv/PnM_positions_std.csv", sep = "\t")
contacts_in_significant_pairsPnM.head()

,chrom,pos1,strand1,h1,pos2,strand2,h2,transh,pos1-1,pos2-1,interactionID,replicate,dist,bin_name1,bin_name2,bin pair type,std1,std2,num_contacts
0,chr2L,10138157,DGRP-439,DGRP-439,10143665,DGRP-439,DGRP-439,False,10138156,10143664,1158040,Rep1,5508,chr2L:10137500-10140000,chr2L:10142500-10145000,transDOWN,103.152546,128.000087,95
1,chr2L,10138118,+,DGRP-439,10143754,-,DGRP-439,False,10138117,10143753,3501883,Rep1,5636,chr2L:10137500-10140000,chr2L:10142500-10145000,transDOWN,103.152546,128.000087,95
2,chr2L,10138009,+,DGRP-439,10143696,-,DGRP-439,False,10138008,10143695,3689934,Rep1,5687,chr2L:10137500-10140000,chr2L:10142500-10145000,transDOWN,103.152546,128.000087,95
3,chr2L,10138139,+,DGRP-439,10143706,-,DGRP-439,False,10138138,10143705,4796396,Rep1,5567,chr2L:10137500-10140000,chr2L:10142500-10145000,transDOWN,103.152546,128.000087,95
4,chr2L,10138003,DGRP-439,DGRP-439,10143646,DGRP-439,DGRP-439,False,10138002,10143645,5125548,Rep1,5643,chr2L:10137500-10140000,chr2L:10142500-10145000,transDOWN,103.152546,128.000087,95


In [15]:
significantPnM = pd.read_csv("../../results/csv/PnM_diff_bin_pairs.csv", sep = "\t", index_col = 0)
significantPnM["comparison"] = significantPnM.index.str.split("_chr").str[0]
significantPnM.head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,comparison
DGRP-57_locus1_chr2L:10875000-10877500_chr2L:10882500-10885000,92.201972,-5.297533,0.539083,-9.826939,8.619868e-23,7.292712e-19,DGRP-57_locus1
DGRP-57_locus1_chr2L:11562500-11565000_chr2L:11567500-11570000,10.378207,-6.625061,1.560225,-4.246221,2.174060e-05,1.082886e-02,DGRP-57_locus1
DGRP-57_locus1_chr2L:11620000-11622500_chr2L:11625000-11627500,16.060058,-2.810612,0.728042,-3.860507,1.131520e-04,2.904185e-02,DGRP-57_locus1
DGRP-57_locus1_chr2L:11625000-11627500_chr2L:11630000-11632500,18.063646,-2.246227,0.620293,-3.621236,2.931986e-04,4.574474e-02,DGRP-57_locus1
DGRP-57_locus1_chr2L:11660000-11662500_chr2L:11665000-11667500,7.568995,-6.169620,1.612010,-3.827284,1.295652e-04,3.100354e-02,DGRP-57_locus1


In [16]:
PnMrep1 = contacts_in_significant_pairsPnM[contacts_in_significant_pairsPnM["replicate"] == "Rep1"]
PnMrep2 = contacts_in_significant_pairsPnM[contacts_in_significant_pairsPnM["replicate"] == "Rep2"]

for num, df in [("1", PnMrep1), ("2", PnMrep2)]:
    for comparison, group in significantPnM.groupby("comparison"):
        cis, trans = prepare_bedpe_files(df, group)
        cis.to_csv("../../results/bed/PnM_%s_cis_Rep%s.bedpe" % (comparison, num), index = False, sep = "\t")
        trans.to_csv("../../results/bed/PnM_%s_trans_Rep%s.bedpe" % (comparison, num), index = False, sep = "\t")

In [17]:
for comparison, group in significantPnM.groupby("comparison"):
    fc_colored = prepare_bedpe_files_foldchange(group)
    fc_colored.to_csv("../../results/bed/PnM_%s_log2FC_colored.bedpe" % comparison, index = False, sep = "\t")